Tuning inside Adagrid is a scary thing to do. This document is a summary of the various problems I've run into. 

First, some basics. We have three different groups of thresholds. $i$ is a tile index, $j$ is a bootstrap index.
1. The original sample, $\lambda^*_i$ and it's grid-wise minimum $\lambda^{**}$. 
2. $N_B$  global bootstraps $\lambda_{i, B_j}^*$ and their grid-wise minima $\lambda_{B_j}^{**}$. In the code, info regarding these bootstraps is prefixed with `B_`.
3. $N_b$  tile-wise investigation bootstraps $\lambda_{i, b_j}^*$ and their tile-wise minima $\lambda_{i}^{**}$. In the code, info regarding these bootstraps is prefixed with `twb_` standing for "tile-wise bootstrap". 

For each of these tuning problems, we tune at TIE level $\alpha_0 = \alpha - C_{\alpha}$ where $C_{\alpha}$ is the TIE consumed by continuous simulation extension. The C stands for "cost" and in the code this is called `alpha_cost`. 

The different problems I've run into so far:
- impossible tuning. This occurs when $\alpha_0 < 2 / (K+1)$ . In this situation, we can't tune because there are too few test statistics. We need to either run more simulations (increase $K$) or refine (increase $\alpha_0$). 
- it's possible to have a tile where the twb_min_lam is large... like 1 but B_lam is small like 0.015. 
	- these tiles have too much variance, but there's no way to detect them because our tilewise bootstrap didn't turn up any evidence of danger. 
	- it's not possible to completely remove this possibility because there's always some randomness.
	- this partially suggests i'm using a baseline of too few simulations or too large tiles. this is fixable. I bumped up the baseline K to 4096.
	- another option would be to use a new bootstrap in some way to get a new sample?
- part of the problem is tiles for which $\alpha_0$ is super small and so the tuning result is like index 2 of the batch which will of course result in a high variance. the simple thing to do is to make $\alpha_0$ larger. is there a smooth way to do this?

In [1]:
import confirm.outlaw.nb_util as nb_util

nb_util.setup_nb(pretty=True)

import time
import jax
import os
import re
import pickle
import numpy as np
import jax.numpy as jnp
import scipy.spatial
import matplotlib.pyplot as plt
from confirm.mini_imprint import grid
from confirm.lewislib import grid as lewgrid
from confirm.lewislib import lewis, batch
from confirm.mini_imprint import binomial, checkpoint

import confirm.mini_imprint.lewis_drivers as lts

from rich import print as rprint

# Configuration used during simulation
name = "4d_full"
params = {
    "n_arms": 4,
    "n_stage_1": 50,
    "n_stage_2": 100,
    "n_stage_1_interims": 2,
    "n_stage_1_add_per_interim": 100,
    "n_stage_2_add_per_interim": 100,
    "stage_1_futility_threshold": 0.15,
    "stage_1_efficacy_threshold": 0.7,
    "stage_2_futility_threshold": 0.2,
    "stage_2_efficacy_threshold": 0.95,
    "inter_stage_futility_threshold": 0.6,
    "posterior_difference_threshold": 0,
    "rejection_threshold": 0.05,
    "key": jax.random.PRNGKey(0),
    "n_table_pts": 20,
    "n_pr_sims": 100,
    "n_sig2_sims": 20,
    "batch_size": int(2**12),
    "cache_tables": f"./{name}/lei_cache.pkl",
}

# Configuration used during simulation
# name = "3d_smaller2"
# params = {
#     "n_arms": 3,
#     "n_stage_1": 50,
#     "n_stage_2": 100,
#     "n_stage_1_interims": 2,
#     "n_stage_1_add_per_interim": 100,
#     "n_stage_2_add_per_interim": 100,
#     "stage_1_futility_threshold": 0.15,
#     "stage_1_efficacy_threshold": 0.7,
#     "stage_2_futility_threshold": 0.2,
#     "stage_2_efficacy_threshold": 0.95,
#     "inter_stage_futility_threshold": 0.6,
#     "posterior_difference_threshold": 0,
#     "rejection_threshold": 0.05,
#     "key": jax.random.PRNGKey(0),
#     "n_table_pts": 20,
#     "n_pr_sims": 100,
#     "n_sig2_sims": 20,
#     "batch_size": int(2**12),
#     "cache_tables": f"./{name}/lei_cache.pkl",
# }

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_arms = params["n_arms"]
ns = np.concatenate(
    [np.ones(n_arms - 1)[:, None], -np.eye(n_arms - 1)],
    axis=-1,
)
null_hypos = [grid.HyperPlane(n, 0) for n in ns]
symmetry = []
for i in range(n_arms - 2):
    n = np.zeros(n_arms)
    n[i + 1] = 1
    n[i + 2] = -1
    symmetry.append(grid.HyperPlane(n, 0))

theta_min = -1.0
theta_max = 1.0
init_grid_size = 8
theta, radii = grid.cartesian_gridpts(
    np.full(n_arms, theta_min),
    np.full(n_arms, theta_max),
    np.full(n_arms, init_grid_size),
)
g_raw = grid.build_grid(theta, radii)
g = grid.build_grid(
    theta, radii, null_hypos=null_hypos, symmetry_planes=symmetry, should_prune=True
)

In [3]:
import adastate
from criterion import Criterion

lei_obj = lewis.Lewis45(**params)
n_arm_samples = int(lei_obj.unifs_shape()[0])

In [4]:
P = adastate.AdaParams(
    init_K=2**11,
    n_K_double=8,
    alpha_target=0.025,
    grid_target=0.002,
    bias_target=0.002,
    nB_global=50,
    nB_tile=50,
    step_size=2**14,
    tuning_min_idx=20
)
D = adastate.init_data(P, lei_obj, 0)
adastate.save(f"./{name}/data_params.pkl", (P, D))

In [7]:
load_iter = 'latest'
S, load_iter, fn = adastate.load(name, load_iter)
if S is None:
    print('initializing')
    S = adastate.init_state(P, g)
S.todo[0] = True

loading checkpoint 4d_full/124.pkl


In [8]:
R = adastate.AdaRunner(P, lei_obj)
iter_max = 10000
cost_per_sim = np.inf
for II in range(load_iter + 1, iter_max):
    if np.sum(S.todo) == 0:
        break

    print(f"starting iteration {II} with {np.sum(S.todo)} tiles to process")
    total_effort = np.sum(S.sim_sizes[S.todo])
    predicted_time = total_effort * cost_per_sim
    print(f"runtime prediction: {predicted_time:.2f}")

    start = time.time()
    R.step(P, S, D)
    cost_per_sim = (time.time() - start) / total_effort
    print(f"step took {time.time() - start:.2f}s")

    start = time.time()
    adastate.save(f"{name}/{II}.pkl", S)
    for old_i in checkpoint.exponential_delete(II, base=1):
        fp = f"{name}/{old_i}.pkl"
        if os.path.exists(fp):
            os.remove(fp)
    print(f"checkpointing took {time.time() - start:.2f}s")

    start = time.time()
    cr = Criterion(lei_obj, P, S, D)
    print(f'criterion took {time.time() - start:.2f}s')
    rprint(cr.report)

    start = time.time()
    if (np.sum(cr.which_refine) > 0 or np.sum(cr.which_deepen) > 0) and II != iter_max - 1:
        S.sim_sizes[cr.which_deepen] = S.sim_sizes[cr.which_deepen] * 2
        S.todo[cr.which_deepen] = True

        S = S.refine(P, cr.which_refine, null_hypos, symmetry)
        print(f"refinement took {time.time() - start:.2f}s")

starting iteration 125 with 1 tiles to process
runtime prediction: inf
tuning for 32768 simulations with 1 tiles and batch size (64, 1024)
0.8734002113342285
0.3879389762878418
step took 3.35s
checkpointing took 7.52s
tuning for 2048 simulations with 1 tiles and batch size (1, 16384)
0.08160567283630371
0.2525291442871094
criterion took 11.08s


{
    'overall_lam': '0.04873',
    'lam_std': '0.0066',
    'grid_cost': '0.01358',
    'bias': '0.00527',
    'total_slack': '0.01885',
    'n_tiles': 11051773,
    'n_refine': 5496,
    'n_refine_impossible': 0,
    'n_moresims': 10888,
    'n_moresims_impossible': 0
}

refinement took 6.32s
starting iteration 126 with 127848 tiles to process
runtime prediction: 29025.09
tuning for 2048 simulations with 116960 tiles and batch size (64, 1024)
96.89662957191467
19.538331747055054
tuning for 4096 simulations with 10888 tiles and batch size (64, 1024)
17.563320875167847
2.470937490463257
step took 137.82s
checkpointing took 7.13s
tuning for 2048 simulations with 1 tiles and batch size (1, 16384)
0.08059406280517578
0.25332045555114746
criterion took 11.71s


{
    'overall_lam': '0.04873',
    'lam_std': '0.0066',
    'grid_cost': '0.01358',
    'bias': '0.00525',
    'total_slack': '0.01883',
    'n_tiles': 11157631,
    'n_refine': 266,
    'n_refine_impossible': 0,
    'n_moresims': 16118,
    'n_moresims_impossible': 0
}

refinement took 7.13s
starting iteration 127 with 22502 tiles to process
runtime prediction: 38.36
tuning for 2048 simulations with 6384 tiles and batch size (64, 1024)
5.336625814437866
1.3505454063415527
tuning for 4096 simulations with 16118 tiles and batch size (64, 1024)
25.87602686882019
3.5235230922698975
step took 36.90s
checkpointing took 7.26s
tuning for 2048 simulations with 1 tiles and batch size (1, 16384)
0.09521079063415527
0.26537060737609863
criterion took 11.09s


...

refinement took 10.00s
starting iteration 148 with 32928 tiles to process
runtime prediction: 47.03
tuning for 2048 simulations with 17600 tiles and batch size (64, 1024)
14.557708501815796
2.837231159210205
tuning for 4096 simulations with 15328 tiles and batch size (64, 1024)
24.573076725006104
3.2899112701416016
step took 46.26s
checkpointing took 8.13s
tuning for 2048 simulations with 1 tiles and batch size (1, 16384)
0.09479641914367676
0.2855570316314697
criterion took 14.07s


{
    'overall_lam': '0.04967',
    'lam_std': '0.0068',
    'grid_cost': '0.01368',
    'bias': '0.00624',
    'total_slack': '0.01992',
    'n_tiles': 13166746,
    'n_refine': 1684,
    'n_refine_impossible': 0,
    'n_moresims': 14700,
    'n_moresims_impossible': 0
}

refinement took 9.48s
starting iteration 149 with 52304 tiles to process
runtime prediction: 64.23
tuning for 2048 simulations with 37604 tiles and batch size (64, 1024)
31.101231336593628
5.832622051239014
tuning for 4096 simulations with 14700 tiles and batch size (64, 1024)
23.548077821731567
3.1711807250976562
step took 64.76s
checkpointing took 8.41s
tuning for 2048 simulations with 1 tiles and batch size (1, 16384)
0.10033369064331055
0.27898335456848145
criterion took 13.21s


{
    'overall_lam': '0.04967',
    'lam_std': '0.0068',
    'grid_cost': '0.01368',
    'bias': '0.00624',
    'total_slack': '0.01992',
    'n_tiles': 13200405,
    'n_refine': 7492,
    'n_refine_impossible': 0,
    'n_moresims': 8892,
    'n_moresims_impossible': 0
}

refinement took 9.51s
starting iteration 150 with 159588 tiles to process
runtime prediction: 162.84
tuning for 2048 simulations with 150696 tiles and batch size (64, 1024)


In [64]:
typeI_sum = batched_rej(
    sim_sizes,
    (np.full(sim_sizes.shape[0], overall_cv),
    g.theta_tiles,
    g.null_truth,),
    unifs,
    unifs_order,
)

savedata = [
    g,
    sim_sizes,
    bootstrap_cvs,
    typeI_sum,
    hob_upper,
    pointwise_target_alpha
]
with open(f"{name}/final.pkl", "wb") as f:
    pickle.dump(savedata, f)

# Calculate actual type I errors?
typeI_est, typeI_CI = binomial.zero_order_bound(
    typeI_sum, sim_sizes, delta_validate, 1.0
)
typeI_bound = typeI_est + typeI_CI

hob_upper = binomial.holder_odi_bound(
    typeI_bound, g.theta_tiles, g.vertices, n_arm_samples, holderq
)
sim_cost = typeI_CI
hob_empirical_cost = hob_upper - typeI_bound
worst_idx = np.argmax(typeI_est)
worst_tile = g.theta_tiles[worst_idx]
typeI_est[worst_idx], worst_tile
worst_cv_idx = np.argmin(sim_cvs)
typeI_est[worst_cv_idx], sim_cvs[worst_cv_idx], g.theta_tiles[worst_cv_idx], pointwise_target_alpha[worst_cv_idx]
plt.hist(typeI_est, bins=np.linspace(0.02,0.025, 100))
plt.show()

theta_0 = np.array([-1.0, -1.0, -1.0])      # sim point
v = 0.1 * np.ones(theta_0.shape[0])     # displacement
f0 = 0.01                               # Type I Error at theta_0
fwd_solver = ehbound.ForwardQCPSolver(n=n_arm_samples)
q_opt = fwd_solver.solve(theta_0=theta_0, v=v, a=f0) # optimal q
ehbound.q_holder_bound_fwd(q_opt, n_arm_samples, theta_0, v, f0)

running for size 1000 with 4721515 tiles took 1573.7245726585388
